In [1]:
import torch
import torch.autograd as autograd

import numpy as np
from pytorchnet import PytorchNet
from tfnet import TFNet
        
# bug: filter reshaping possibly causes incorrect results 
class ConvBenchmark:
    def __init__(self, **bench_args):
        convweights = bench_args["convweights"]
        self.conv_filters_ = []
        #filter_ = autograd.Variable(torch.FloatTensor([[[[-1, 1]]]])).numpy()
        for w in convweights:
            w_array = np.random.random(w).astype(np.float32)
            self.conv_filters_.append(w_array)
            #self.conv_filters_.append(filter_)
        
        
        if "poolings" in bench_args:
            self.poolings_ = bench_args["poolings"]
        else:
            self.poolings_ = False
        
        self.networks_ = {}
        self.networks_["pytorchCNN"] = PytorchNet(self.conv_filters_, self.poolings_)
        self.networks_["TF_CNN"] = TFNet(self.conv_filters_, self.poolings_)
        # TODO: self.networks_["CNTK_CNN"] = TFNet(self.conv_filters_, self.poolings_)
        
        self.input_ = np.random.random(bench_args["inputsize"]).astype(np.float32)
        #self.input_ = autograd.Variable(torch.FloatTensor([[[[0,0,0,0,0,0,0], [0, 0, 1, 1, 1, 0, 0], 
        #                                     [0, 0, 1, 1, 1, 0, 0], [0, 0, 1, 1, 1, 0, 0],
        #                                    [0,0,0,0,0,0,0]]]])).numpy()
        

        self.results_ = {}
        
    def runNetworks(self):
        # Pytorch nn.Conv2d takes input a 4D Tensor of [nSamples x nChannels x Height x Width]
        # Input is defined in Pytorch layout fashion
        for netname, network in self.networks_.items():
            #output, t = network.forward(np.copy(self.input_))
            #self.results_[netname] = {"output": output, "runtime": t}
            output, t, conv_filters = network.forward(np.copy(self.input_))
            self.results_[netname] = {"output": output, "runtime": t, "filters": conv_filters}

            print("{} took {:.3} seconds.".format(netname, t))
            
        

In [2]:
inBS, inCh, inHeight, inWidth = (2,1,32,32)
inputsize = inBS, inCh, inHeight, inWidth

# Filters are 4D tensor in form [out_channels, in_channels, fil_height, fil_width]
conv1w = (6, 1, 5, 5)
conv2w = (16, 6, 5, 5)
convweights = (conv1w, conv2w)

poolings = (2,2)

benchmark = ConvBenchmark(inputsize=inputsize, convweights=convweights, poolings=poolings)

In [3]:
#benchmark = ConvBenchmark(inputsize=(1,1,5,7), convweights=((2, 1, 3, 3),), poolings=(2,1))

In [4]:
benchmark.runNetworks()

layer: 0
size of x before: torch.Size([2, 1, 32, 32])
size of filter : torch.Size([1, 1, 1, 2])
size of x after filter: torch.Size([2, 1, 32, 31])
size of x after pool: torch.Size([2, 1, 16, 15])
 
layer: 1
size of x before: torch.Size([2, 1, 16, 15])
size of filter : torch.Size([1, 1, 1, 2])
size of x after filter: torch.Size([2, 1, 16, 14])
size of x after pool: torch.Size([2, 1, 8, 7])
 
pytorchCNN took 0.0342 seconds.
layer: 0
size of x before: (2, 32, 32, 1)
size of filter : (1, 2, 1, 1)
size of x after filter: (2, 32, 31, 1)
size of x after pool: (2, 16, 15, 1)
 
layer: 1
size of x before: (2, 16, 15, 1)
size of filter : (1, 2, 1, 1)
size of x after filter: (2, 16, 14, 1)
size of x after pool: (2, 8, 7, 1)
 
TF_CNN took 0.034 seconds.


In [5]:
pytorch_out = benchmark.results_["pytorchCNN"]["output"]
tf_out = benchmark.results_["TF_CNN"]["output"]

In [6]:
#tf_out = tf_out.reshape(pytorch_out.shape)
print(tf_out.shape)
print(type(tf_out))

(2, 8, 7, 1)
<class 'numpy.ndarray'>


In [7]:
print(pytorch_out.shape)
print(type(pytorch_out))

(2, 1, 8, 7)
<class 'numpy.ndarray'>


In [8]:
pytorch_out

array([[[[ 2.67966318,  2.71471167,  3.09804201,  3.14122438,  2.81473494,
           2.96181679,  2.58565712],
         [ 3.12691641,  2.1866827 ,  2.7761898 ,  2.26824212,  2.73713684,
           2.81289315,  2.91775513],
         [ 3.12841392,  2.89322925,  3.01870632,  3.053334  ,  2.84481311,
           2.82143736,  2.68490744],
         [ 2.96692824,  2.95226693,  3.13295245,  2.92523575,  2.81548882,
           2.28427792,  2.45197725],
         [ 2.70435309,  2.93887663,  2.17372704,  2.8330605 ,  2.66174412,
           2.73455477,  2.72392917],
         [ 2.97445297,  2.6484437 ,  2.73997498,  2.8487134 ,  3.0648818 ,
           2.79223418,  2.95389986],
         [ 2.64321113,  2.6719017 ,  2.65850353,  2.86545253,  3.06777239,
           2.53444052,  3.12648726],
         [ 2.39312887,  2.61828184,  2.46704364,  2.68779182,  2.52999473,
           2.65342593,  2.6569376 ]]],


       [[[ 2.55437446,  2.18620157,  2.92404032,  2.51848865,  3.03364325,
           2.94478345,  2

In [9]:
tf_out

array([[[[ 2.67966318],
         [ 2.71471167],
         [ 3.09804201],
         [ 3.14122438],
         [ 2.81473494],
         [ 2.96181679],
         [ 2.58565712]],

        [[ 3.12691641],
         [ 2.1866827 ],
         [ 2.7761898 ],
         [ 2.26824212],
         [ 2.73713684],
         [ 2.81289315],
         [ 2.91775513]],

        [[ 3.12841392],
         [ 2.89322925],
         [ 3.01870632],
         [ 3.053334  ],
         [ 2.84481311],
         [ 2.82143736],
         [ 2.68490744]],

        [[ 2.96692824],
         [ 2.95226693],
         [ 3.13295245],
         [ 2.92523575],
         [ 2.81548882],
         [ 2.28427792],
         [ 2.45197725]],

        [[ 2.70435309],
         [ 2.93887663],
         [ 2.17372704],
         [ 2.8330605 ],
         [ 2.66174412],
         [ 2.73455477],
         [ 2.72392917]],

        [[ 2.97445297],
         [ 2.6484437 ],
         [ 2.73997498],
         [ 2.8487134 ],
         [ 3.0648818 ],
         [ 2.79223418],
      

In [10]:
#atol: abs tolerance
#np.isclose([1.777774, 1.0003], [1.777772, 1.0001], atol=0.001) -> [True False]
#np.isclose([1.777774, 1.0003], [1.777772, 1.01], atol=0.001).all() -> False
tf_out = tf_out.reshape(pytorch_out.shape)
np.isclose(tf_out, pytorch_out, atol=0.001).all()

True

In [11]:
pytorch_fils = benchmark.results_["pytorchCNN"]["filters"]
tf_fils = benchmark.results_["TF_CNN"]["filters"]

In [12]:
pytorch_fils

[tensor([[[[-0.3103,  1.8064]]]]), tensor([[[[-0.2114,  1.9227]]]])]

In [13]:
tf_fils

[array([[[[-0.31027031]],
 
         [[ 1.80641842]]]], dtype=float32), array([[[[-0.21142304]],
 
         [[ 1.92265344]]]], dtype=float32)]

In [14]:
        # TF conv.2d takes filter as a 4D Tensor of  [fil_height, fil_width, in_channels, out_channels] 
len(tf_fils[0][0])

2

In [15]:
import torch
import torch.autograd as autograd




In [16]:
filters = autograd.Variable(torch.FloatTensor([[[[-1, 1]]]])).numpy()

In [17]:
input_tensor = autograd.Variable(torch.FloatTensor([[[[0,0,0,0,0,0,0], [0, 0, 1, 1, 1, 0, 0], 
                                             [0, 0, 1, 1, 1, 0, 0], [0, 0, 1, 1, 1, 0, 0],
                                            [0,0,0,0,0,0,0]]]])).numpy()